In [1]:
!pip install firebase-admin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 14.1 MB/s eta 0:00:00


In [5]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

service_account_path = '/kaggle/input/firebase-m-vet/serviceAccountKey.json'

cred = credentials.Certificate(service_account_path)

# Check if the default app has already been initialized
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)

db = firestore.client()

def fetch_animal_data():
    animal_officers_ref = db.collection('animal_officer')
    officers = animal_officers_ref.stream()

    all_animal_data = []

    for officer in officers:
        animals_ref = db.collection('animal_officer').document(officer.id).collection('animals')
        animals = animals_ref.stream()

        for animal in animals:
            clinical_symptoms = animal.get('clinicalSymptoms')
            if clinical_symptoms is None:
                clinical_symptoms = []
            animal_data = {
                'Animal ID': animal.id,
                'Age': animal.get('age'),
                'Breed': animal.get('breed'),
                'Body Posture': animal.get('bodyPosture'),
                'Body Score': animal.get('bodyScore'),
                'Clinical Symptoms': ", ".join(clinical_symptoms)
            }
            all_animal_data.append(animal_data)
    
    return all_animal_data

animal_data = fetch_animal_data()
df = pd.DataFrame(animal_data)

excel_filename = 'animal_data.xlsx'

df.to_excel(excel_filename, index=False)

print(f'Data successfully saved to {excel_filename}')


Data successfully saved to animal_data.xlsx
